In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
import logging
logging.disable(logging.WARNING)


In [3]:
final_train_df = pd.read_csv('../data/final_train_df.csv')
final_test_df = pd.read_csv('../data/final_test_df.csv')


In [4]:
train_df, val_df = train_test_split(final_train_df, test_size=0.1, random_state=42)

print(f"Размер обучающей выборки: {train_df.shape}")
print(f"Размер валидационной выборки: {val_df.shape}")
print(f"Доступно GPU: {torch.cuda.device_count()}")


Размер обучающей выборки: (89964, 3)
Размер валидационной выборки: (9996, 3)
Доступно GPU: 2


In [5]:
class TextPairDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        """
        Инициализация датасета.
        
        Параметры:
        - df: pandas DataFrame с колонками 'text1', 'text2', 'score'.
        - tokenizer: токенизатор из библиотеки Hugging Face.
        - max_length: максимальная длина токенизированного ввода.
        """
        self.texts1 = df['text1'].tolist()
        self.texts2 = df['text2'].tolist()
        self.scores = df['score'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.scores)
    
    def __getitem__(self, idx):
        text1 = self.texts1[idx]
        text2 = self.texts2[idx]
        score = self.scores[idx]
        
        # Токенизация пары текстов
        encoding = self.tokenizer(
            text1,
            text2,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Возвращаем словарь с токенами и меткой
        return {
            'input_ids': encoding['input_ids'].squeeze(),  # Убираем лишнюю размерность
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(score, dtype=torch.float)
        }

In [6]:
# Выбор предобученной модели и токенизатора
model_name = 'bert-base-multilingual-cased'  # Поддерживает русский язык
tokenizer = BertTokenizer.from_pretrained(model_name)

# Загрузка модели для классификации с одним выходом (регрессия)
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,  # Один выход для регрессии
    problem_type="regression"
)

In [7]:
# Функция для вычисления метрик
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    rmse = np.sqrt(mean_squared_error(labels, predictions))
    mae = mean_absolute_error(labels, predictions)
    return {
        'rmse': rmse,
        'mae': mae
    }

# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='rmse',
    greater_is_better=False
)


# Создание датасетов
train_dataset = TextPairDataset(train_df, tokenizer)
val_dataset = TextPairDataset(val_df, tokenizer)

C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Ранняя остановка
)

# Обучение модели
trainer.train()

C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")


Epoch,Training Loss,Validation Loss,Rmse,Mae
1,0.076100,0.089826,0.299709,0.185356
2,0.064400,0.070381,0.265294,0.200273
3,0.041100,0.059197,0.243304,0.169772
4,0.040700,0.052689,0.229542,0.161219
5,0.024700,0.070806,0.266094,0.203038
6,0.020000,0.068782,0.262264,0.210278


C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
C:\Users\q\anaconda3\envs\pyto

TrainOutput(global_step=8436, training_loss=0.14808191039858695, metrics={'train_runtime': 20280.3434, 'train_samples_per_second': 44.36, 'train_steps_per_second': 0.693, 'total_flos': 3.550546568449843e+16, 'train_loss': 0.14808191039858695, 'epoch': 6.0})

In [9]:
# Получение результатов оценки
metrics = trainer.evaluate()

print("Метрики на валидационной выборке:")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key}: {value:.4f}")

C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Метрики на валидационной выборке:
eval_loss: 0.0527
eval_rmse: 0.2295
eval_mae: 0.1612
eval_runtime: 171.7535
eval_samples_per_second: 58.2000
eval_steps_per_second: 0.9140


In [10]:
# Создание датасета для тестовых данных
test_dataset = TextPairDataset(final_test_df, tokenizer)

# Предсказание
predictions = trainer.predict(test_dataset)

# Извлечение предсказанных оценок
pred_scores = predictions.predictions.squeeze()

# Добавление предсказанных оценок в DataFrame
final_test_df['pred_score'] = pred_scores

# Просмотр первых строк
print(final_test_df.head())

# Сохранение результатов
final_test_df.to_csv('../data/final_test_with_predictions.csv', index=False)

C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


                                               text1  \
0  отвечать работа партия внести свой вклад 1936 ...   
1  сложный модель поддержка шифрование данные аут...   
2  это время отплыть два лодка лодка сам генерал ...   
3  первый юридически обязывать добровольный докум...   
4  длина передний крыло 4552 мм размах крыло 1001...   

                                               text2     score  pred_score  
0  внести свой вклад урегулирование инцидент сиан...  4.889081    4.997534  
1  сложный модель обладать функция шифрование аут...  4.865897    4.644709  
2  два лодка отправиться плавание лодка генерал у...  4.316391    4.484791  
3  решение 18551863 добровольный вхождение киргиз...  4.767547    5.026204  
4  передний крыло иметь длина 4552 мм размах крыл...  4.839230    4.906640  


In [11]:
# Сохранение модели и токенизатора
model.save_pretrained('../model/trained_model')
tokenizer.save_pretrained('../model/trained_model')


('../model/trained_model\\tokenizer_config.json',
 '../model/trained_model\\special_tokens_map.json',
 '../model/trained_model\\vocab.txt',
 '../model/trained_model\\added_tokens.json')

In [12]:
final_test_df

,text1,text2,score,pred_score
0,отвечать работа партия внести свой вклад 1936 ...,внести свой вклад урегулирование инцидент сиан...,4.889081,4.997534
1,сложный модель поддержка шифрование данные аут...,сложный модель обладать функция шифрование аут...,4.865897,4.644709
2,это время отплыть два лодка лодка сам генерал ...,два лодка отправиться плавание лодка генерал у...,4.316391,4.484791
3,первый юридически обязывать добровольный докум...,решение 18551863 добровольный вхождение киргиз...,4.767547,5.026204
4,длина передний крыло 4552 мм размах крыло 1001...,передний крыло иметь длина 4552 мм размах крыл...,4.839230,4.906640
...,...,...,...,...
19994,болтон предполагать конгрессмен демократ генри...,мнение генри уоксмэн болтон повлиять решение б...,4.524049,4.499201
19995,дом 11с1 тип здание спортивный сооружение этаж...,спортивный сооружение дом 11с1 находиться жило...,4.702792,4.649830
19996,пермь 1909 год заимка 1911 год пермьзаимка жел...,пермь это обычный название жилой район пермь ж...,4.401115,3.718822
19997,оконце жизнь оставаться президент публиковать ...,недолго пробыть пост президент компания выпуск...,3.778666,3.885373


In [14]:
min_score = final_test_df['score'].min()
max_score = final_test_df['score'].max()

min_pred_score = final_test_df['pred_score'].min()
max_pred_score = final_test_df['pred_score'].max()

print(f"Диапазон значений в столбце score: {min_score} - {max_score}")
print(f"Диапазон значений в столбце pred_score: {min_pred_score} - {max_pred_score}")


Диапазон значений в столбце score: 0.7129944860935211 - 5.000000596046448
Диапазон значений в столбце pred_score: 1.615647554397583 - 5.049973964691162
